In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc

In [3]:
data_path = "../../allData/validationData/out_7day_test/test_sessions.jsonl"

In [27]:
%%time
test_df = pd.DataFrame()
chunks = pd.read_json(data_path, lines=True, chunksize=100_000)

for chunk in chunks:
    event_dict = {'session': [],'aid': [],'ts': [],'type': []}
    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            event_dict['ts'].append(event['ts'])
            event_dict['type'].append(event['type'])
    chunk_session = pd.DataFrame(event_dict)
    test_df = pd.concat([test_df, chunk_session])
            
test_df = test_df.reset_index(drop=True)
test_df

CPU times: user 37 s, sys: 4.55 s, total: 41.5 s
Wall time: 46.1 s


,session,aid,ts,type
0,11098528,11830,1661119200060,clicks
1,11098529,1105029,1661119200259,clicks
2,11098530,264500,1661119200974,clicks
3,11098530,264500,1661119288407,clicks
4,11098530,409236,1661119369986,clicks
...,...,...,...,...
7580963,12899774,33035,1661723968869,clicks
7580964,12899775,1743151,1661723970935,clicks
7580965,12899776,548599,1661723972537,clicks
7580966,12899777,384045,1661723976974,clicks


In [7]:
%%time
test_df["minutes"] = test_df[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)
test_df = test_df.sort_values(['minutes'],ascending=False)

test_action_df = test_df.copy()
test_action_df.aid = ' ' + test_df.aid.astype('str')
test_action_df = test_action_df.groupby(['session','type'])['aid'].sum().reset_index()
test_action_df

CPU times: user 3min 45s, sys: 1min 16s, total: 5min 2s
Wall time: 5min 33s


,session,type,aid
0,11098528,clicks,11830
1,11098529,clicks,1105029
2,11098530,carts,409236
3,11098530,clicks,409236 409236 264500 264500 409236
4,11098531,clicks,1365569 1553691 1271998 1309633 452188 624163...
...,...,...,...
2086659,12899774,clicks,33035
2086660,12899775,clicks,1743151
2086661,12899776,clicks,548599
2086662,12899777,clicks,384045


In [8]:
next_orders_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'carts'), ])
next_orders_df['type'] = 'orders'
next_orders_df

,session,type,aid
2,11098530,orders,409236
7,11098533,orders,1074173 1309900
10,11098535,orders,745365
13,11098537,orders,1409748 1409748 358965
16,11098538,orders,1711586
...,...,...,...
2086483,12899601,orders,946970
2086495,12899613,orders,983198
2086514,12899631,orders,1055835
2086528,12899644,orders,436912 822934


In [9]:
next_carts_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ])
next_carts_df['type'] = 'carts'
next_carts_df

,session,type,aid
0,11098528,carts,11830
1,11098529,carts,1105029
3,11098530,carts,409236 409236 264500 264500 409236
4,11098531,carts,1365569 1553691 1271998 1309633 452188 624163...
6,11098532,carts,7651 876469
...,...,...,...
2086659,12899774,carts,33035
2086660,12899775,carts,1743151
2086661,12899776,carts,548599
2086662,12899777,carts,384045


In [10]:
next_clicks_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ]).copy()

In [11]:
recommend_df = pd.concat([next_orders_df, next_carts_df, next_clicks_df], axis =0)
recommend_df["session_type"] = recommend_df["session"].astype('str') + "_" + recommend_df["type"] 
recommend_df

,session,type,aid,session_type
2,11098530,orders,409236,11098530_orders
7,11098533,orders,1074173 1309900,11098533_orders
10,11098535,orders,745365,11098535_orders
13,11098537,orders,1409748 1409748 358965,11098537_orders
16,11098538,orders,1711586,11098538_orders
...,...,...,...,...
2086659,12899774,clicks,33035,12899774_clicks
2086660,12899775,clicks,1743151,12899775_clicks
2086661,12899776,clicks,548599,12899776_clicks
2086662,12899777,clicks,384045,12899777_clicks


In [16]:
clicks_template = test_df[["session"]]
clicks_template["session_type"] = clicks_template["session"].astype('str') + "_clicks"
clicks_template = clicks_template["session_type"]

carts_template = test_df[["session"]]
carts_template["session_type"] = carts_template["session"].astype('str') + "_carts"
carts_template = carts_template["session_type"]

orders_template = test_df[["session"]]
orders_template["session_type"] = orders_template["session"].astype('str') + "_orders"
orders_template = orders_template["session_type"]

sub_template = pd.concat([clicks_template, carts_template, orders_template], axis =0)

gc.collect()

<ipython-input-16-b08c6f60f7dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clicks_template["session_type"] = clicks_template["session"].astype('str') + "_clicks"
<ipython-input-16-b08c6f60f7dd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carts_template["session_type"] = carts_template["session"].astype('str') + "_carts"
<ipython-input-16-b08c6f60f7dd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

424

In [18]:
%%time
sample_sub = pd.merge(sub_template, recommend_df[["session_type","aid"]], on = "session_type", how ="left")

In [20]:
best_sold_list = ' 80222 1022566 166037 1629608 1733943 332654 351335 923948 1603001 544144 1083665 832192 29735 231487 563117 247240 673407 125278 800391 527209'

In [23]:
sample_sub['next'] = sample_sub['aid'] + best_sold_list
sample_sub['next'].fillna(best_sold_list, inplace = True)
sample_sub['next'] = sample_sub['next'].str.strip()
sample_sub = sample_sub.drop(["aid"], axis = 1)
sample_sub.columns = ("session_type", "labels")

In [24]:
sample_sub

,session_type,labels
0,11099878_clicks,146447 24584 1649665 80222 1022566 166037 1629...
1,11099345_clicks,1440304 40518 1532894 40518 40518 80222 102256...
2,11102091_clicks,11611 1509028 1316215 1208423 1010831 1809992 ...
3,11099650_clicks,475747 1410944 1539963 1338882 1085520 1410944...
4,11104058_clicks,744282 744282 981486 80222 1022566 166037 1629...
...,...,...
22742899,12899774_orders,80222 1022566 166037 1629608 1733943 332654 35...
22742900,12899775_orders,80222 1022566 166037 1629608 1733943 332654 35...
22742901,12899776_orders,80222 1022566 166037 1629608 1733943 332654 35...
22742902,12899777_orders,80222 1022566 166037 1629608 1733943 332654 35...


In [25]:
sample_sub.to_csv('../../allData/validationData/baseline_sub.csv', index=False)

In [26]:
test_df

,session,aid,ts,type,minutes
10042,11099878,146447,1661119725688,clicks,10061.680233
7121,11099345,1440304,1661119486645,clicks,10043.142233
22414,11102091,11611,1661120880761,clicks,10016.618600
8830,11099650,475747,1661119622543,clicks,10012.681883
32614,11104058,744282,1661123067060,clicks,10000.268083
...,...,...,...,...,...
7580963,12899774,33035,1661723968869,clicks,NaN
7580964,12899775,1743151,1661723970935,clicks,NaN
7580965,12899776,548599,1661723972537,clicks,NaN
7580966,12899777,384045,1661723976974,clicks,NaN


In [ ]:
# < * means visited and unvisited will have different formulas
# *slot_0: is_prev_interacted, 
# slot_1: seq_w_total, 
# slot_2: time_w_total, 
# slot_3: action_w_total: for visited item, ops weight total in this session; for unvisited item, ops weight total of the item referencing this item. 
# slot_4: session_len, 
# slot_5: num uniuqe aids, 
# slot_6: CF_score, 
# slot_7: aid's itemTotalLike: total like score use for normalization.  
# *slot_8: reference time by similar matrix(if aid visited, default 100; if aid not visited, 1-19(when all aid only interact once, could grow to very large if a lot of actions on one aid), depending how many aid reference this item)
# *slot_9: max_sim_score:  (1 if it's a visited item)
# *slot_10: mean_sim_score: (1 if it's a visited item)
# *slot_11: num_interact, (0 for unvisited item; count of interaction for visited item)
# slot_12: time_span of the session 
# slot_13: action_recency: time to last action(end time), for unvisited items -> the time to of reference_aid to the last action
# slot_14: seq_w_max: 
# slot_15: seq_w_mean: for visited item -> seq_w_total / num_interact; for unvisited item -> seq_w_total / reference_time
## ========================= round 2 ================
# slot_16: seq_w_min: 
# slot_17: time_w_max:
# slot_18: time_w_mean: similar to slot_15
# slot_19: time_w_min
# slot_20: ops_w_max:
# slot_21: ops_w_mean:
# slot_22: ops_w_min: 
# slot_23: num_clicks: visited item, direct num; unvisited item, take the reference item's num
# slot_24: num_carts:
# slot_25: num_orders:
# slot_26: last_action_type: 0 -> clicks, 1-> carts, 2 -> orders
# slot_27: time_to_now: latest interaction time to now 
# slot_28: cf_increment_max: 
# slot_29: cf_increment_mean:
# slot_30: cf_increment_min:
##  ======================== Derivable ============================
# slot_31: seq_w_std: slot_14 - slot_16 
# slot_32: time_w_std: slot_17 - slot_19
# slot_33: ops_w_std: slot_20 - slot_22
# slot_34: cf_incre_std: slot_28 - slot_30
## ====================== Round 3 features ========================
# slot_35: last3Inter_cf_incre_max: maximum cf_incre in the last 3 action, if <= 3 actions, same as slot_28
# slot_36: last3Inter_cf_incre_mean: 
# slot_37: last3Inter_cf_incre_min:
# slot_38: last3Inter_time_w_max:  maximum time_w in the last 3 action, if <= 3 actions, same as slot_17
# slot_39: last3Inter_time_w_mean:
# slot_40: last3Inter_time_w_min: 
# slot_41: last3Inter_seq_w_max:  maximum time_w in the last 3 action, if <= 3 actions, same as slot_14
# slot_42: last3Inter_seq_w_mean:
# slot_43: last3Inter_seq_w_min:
# slot_44: raw_seq_order: last action's seq_order, no depreciation
# slot_45: raw_seq_order_max: 
# slot_46: raw_seq_order_mean:
# slot_47: raw_seq_order_min

# slot_48: append ts lastly to unblock feature below
#  

## ================= numba restriction, add in notebook =====================
# slot_49: last_interact_local_day_of_week, Mon -> 0; Tues -> 1; ..... ; Sun -> 6
# slot_50: last_interact_local_hour: 
# slot_51: last_interact_day_night, local ts, if 18:00:00 ~ 2:59:59 -> night/2; 3:00:00 ~ 11:59:59 -> morning/1; 12:00:00 ~ 17:59:59 -> afternoon/0
# >